In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 데이터를 불러옵니다. 파일 경로는 실제 데이터 파일의 경로에 맞게 수정하세요.
data_train = pd.read_csv('/content/drive/MyDrive/open/train.csv')
data_test = pd.read_csv('/content/drive/MyDrive/open/test.csv')

#test DATA feature만 가진 train DATA
train = data_train[data_test.columns.to_list()+['Y_LABEL','SAMPLE_TRANSFER_DAY']].copy()

# 1. 분석의 목적과 목적에 맞는 변수 확인
# 데이터의 구조와 변수 목록을 살펴봅니다.
print("데이터 정보:")
print(data_train.info())

# 데이터의 처음 몇 개 레코드를 확인하여 변수 및 데이터의 형태를 이해합니다.
print("\n데이터 샘플:")
print(data_train.head())

# 2. 데이터형 확인 및 데이터 오류/누락 여부 확인
# 각 열의 데이터 유형 및 결측치 확인
print("\n데이터 유형 및 결측치:")
print(data_train.dtypes)
print(data_train.isnull().sum())

# 3. 데이터 분포 확인
# 데이터의 기술 통계량을 계산하여 분포를 확인합니다.
print("\n데이터 기술 통계량:")
print(data_train.describe())

데이터 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14095 entries, 0 to 14094
Data columns (total 54 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   14095 non-null  object 
 1   COMPONENT_ARBITRARY  14095 non-null  object 
 2   ANONYMOUS_1          14095 non-null  float64
 3   YEAR                 14095 non-null  int64  
 4   SAMPLE_TRANSFER_DAY  14095 non-null  int64  
 5   ANONYMOUS_2          14095 non-null  float64
 6   AG                   14095 non-null  int64  
 7   AL                   14095 non-null  int64  
 8   B                    14095 non-null  int64  
 9   BA                   14095 non-null  int64  
 10  BE                   14095 non-null  int64  
 11  CA                   14095 non-null  int64  
 12  CD                   12701 non-null  float64
 13  CO                   14095 non-null  int64  
 14  CR                   14095 non-null  int64  
 15  CU                   14095 n

### 결측치

In [ ]:
columns_with_missing_values_52 = data_train.columns[data_train.isna().any()].tolist()
columns_with_missing_values_18 = train.columns[train.isna().any()].tolist()


# 출력
print("결측치를 가지고 있는 열:", columns_with_missing_values_52)
print("결측치를 가지고 있는 열:", columns_with_missing_values_18)


결측치를 가지고 있는 열: ['CD', 'FH2O', 'FNOX', 'FOPTIMETHGLY', 'FOXID', 'FSO4', 'FTBN', 'FUEL', 'K', 'SOOTPERCENTAGE', 'U100', 'U75', 'U50', 'U25', 'U20', 'U14', 'U6', 'U4', 'V100']
결측치를 가지고 있는 열: []


In [ ]:
# 결측치가 있는 변수 및 결측률 파악 - 결측률이 0.7이상인 column 17개 제거 ?

mis_val = data_train.isnull().sum()
mis_val_bool = mis_val >= 1
mis_val_df = pd.concat([mis_val, mis_val_bool], axis = 1)
mis_val_df = mis_val_df.rename(columns = {0 : 'mis_val', 1 : 'mis_val_bool'})

mis_val_data = mis_val_df.loc[mis_val_df['mis_val_bool'] == True, :]
mis_val_data['ratio'] = mis_val_data['mis_val'] / 14095
mis_val_data_sorted = mis_val_data.sort_values(by='ratio', ascending=True)
mis_val_data_sorted

<ipython-input-16-310b3d738ff5>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mis_val_data['ratio'] = mis_val_data['mis_val'] / 14095


,mis_val,mis_val_bool,ratio
CD,1394,True,0.098900
K,2299,True,0.163107
FUEL,10205,True,0.724016
FTBN,10205,True,0.724016
FSO4,10205,True,0.724016
SOOTPERCENTAGE,10205,True,0.724016
FOPTIMETHGLY,10205,True,0.724016
FNOX,10205,True,0.724016
FH2O,10205,True,0.724016
FOXID,10205,True,0.724016


In [ ]:
# 결측률이 20% 이상인 변수들 제거

notnull_columns = data_train.loc[:, data_train.notnull().sum(axis = 0) == 14095].columns
notnull_columns = notnull_columns[1 :]
notnull_columns = list(notnull_columns)

select_columns = notnull_columns[: 32] + ['K', 'CD'] + notnull_columns[32 :]
data_train2 = data_train.loc[:, select_columns]
print(data_train2.shape)
print(data_train2.columns)

(14095, 36)
Index(['COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR', 'SAMPLE_TRANSFER_DAY',
       'ANONYMOUS_2', 'AG', 'AL', 'B', 'BA', 'BE', 'CA', 'CO', 'CR', 'CU',
       'FE', 'H2O', 'LI', 'MG', 'MN', 'MO', 'NA', 'NI', 'P', 'PB', 'PQINDEX',
       'S', 'SB', 'SI', 'SN', 'TI', 'V', 'V40', 'K', 'CD', 'ZN', 'Y_LABEL'],
      dtype='object')


### 변수 별 상관분석

In [ ]:
# |0.7|>corelation 인경우 높은 상관관계라고 판단.  -> test data에서 drop할 column은 없다 ?
t = train.copy()
corr = t.corr()
corr.style.background_gradient(cmap = 'coolwarm')

<ipython-input-18-73885dae9c49>:3: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = t.corr()


,ANONYMOUS_1,YEAR,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN,Y_LABEL,SAMPLE_TRANSFER_DAY
ANONYMOUS_1,1.000000,0.106546,0.072154,-0.025674,-0.003783,-0.007035,-0.014086,0.000440,0.003868,-0.004212,-0.006206,-0.007862,0.002277,0.001832,0.001883,0.019542,-0.019991,0.003938,-0.041395
YEAR,0.106546,1.000000,0.138199,-0.129124,-0.052337,-0.028871,-0.137593,-0.057709,0.009855,-0.039132,-0.091482,-0.049206,-0.067160,0.006399,-0.027624,-0.051584,0.048572,-0.053321,-0.094086
ANONYMOUS_2,0.072154,0.138199,1.000000,-0.006266,-0.000070,-0.001826,-0.002244,-0.004608,-0.003760,0.003304,-0.008465,-0.003914,-0.010409,-0.000981,-0.006469,-0.025339,0.032758,-0.000547,-0.000060
AG,-0.025674,-0.129124,-0.006266,1.000000,0.008861,0.005397,0.051046,0.025959,-0.003850,0.019507,0.013124,0.054014,0.030929,0.003609,-0.003853,0.013592,0.002631,0.024032,0.021211
CO,-0.003783,-0.052337,-0.000070,0.008861,1.000000,0.173352,0.265347,0.536765,0.059836,0.425149,0.001389,0.402024,0.174601,0.263407,0.271112,0.107868,-0.018264,0.069203,0.002195
CR,-0.007035,-0.028871,-0.001826,0.005397,0.173352,1.000000,0.050596,0.314226,0.037587,0.247603,-0.008194,0.221551,0.147009,0.226221,0.391048,0.079090,-0.053180,0.133886,-0.001495
CU,-0.014086,-0.137593,-0.002244,0.051046,0.265347,0.050596,1.000000,0.287277,-0.001719,0.222300,-0.017165,0.486860,0.120772,0.102578,0.089311,-0.067575,0.040869,0.096721,0.006303
FE,0.000440,-0.057709,-0.004608,0.025959,0.536765,0.314226,0.287277,1.000000,0.118843,0.622415,-0.063966,0.583322,0.427582,0.384018,0.341445,0.247786,-0.155119,0.352592,0.011651
H2O,0.003868,0.009855,-0.003760,-0.003850,0.059836,0.037587,-0.001719,0.118843,1.000000,0.144325,-0.009545,0.051752,0.081923,0.153990,0.048667,0.371215,-0.031378,0.073294,-0.007926
MN,-0.004212,-0.039132,0.003304,0.019507,0.425149,0.247603,0.222300,0.622415,0.144325,1.000000,-0.047064,0.527618,0.376253,0.585042,0.375177,0.149613,-0.043602,0.230739,0.006816


### IQR
Wilcoxon 순위합 검정을 통해 [이상 집단이 정상 집단보다 큰 값을 가진다]라는 귀무가설에 대한 비모수 검정을 실시했습니다.

p-value ≤ 유의수준 (예: 0.05): 귀무 가설을 기각하고 대립 가설을 채택합니다. 이는 표본 간의 차이가 통계적으로 유의미하다는 것을 의미합니다.

In [ ]:
from scipy.stats import ranksums

def IQR_outlier(data) :
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)

    IQR = Q3 - Q1

    lower_bound = Q1 - (1.5 * IQR)
    upper_bound = Q3 + (1.5 * IQR)

    return pd.concat([lower_bound, upper_bound], axis = 1).T

# 정상 범위와 이상 범위의 데이터 간 윌콕슨 순위합 검정 실시

ranksum_p = []
variable_17 = ['SAMPLE_TRANSFER_DAY', 'AL', 'B', 'BA', 'BE', 'CA', 'CD', 'K', 'LI', 'MG', 'NA', 'P', 'PB', 'S', 'SB', 'SI', 'SN']

# alternative = 'greater'을 걸어줘서 한쪽 검정으로 실시
# scipy 1.8.1 버전 이상이어야 ranksums의 옵션인 alternative(한쪽 검정) 사용 가능
for v in variable_17 :
  temp = ranksums(data_train.loc[(data_train['Y_LABEL'] == 1) & (data_train[v].notnull()), v],
                  data_train.loc[(data_train['Y_LABEL'] == 0) & (data_train[v].notnull()), v],
                  alternative = 'greater').pvalue
  ranksum_p.append(temp)

Wilcoxon_var_table = pd.DataFrame({'variable' : variable_17,
                                   'p_value' : ranksum_p,
                                   'p_value_round' : np.round(ranksum_p, 4)})
Wilcoxon_var_table.sort_values('p_value')


,variable,p_value,p_value_round
15,SI,0.000000e+00,0.0000
13,S,1.970075e-120,0.0000
7,K,1.599942e-50,0.0000
10,NA,5.842989e-36,0.0000
8,LI,2.692208e-09,0.0000
1,AL,2.754589e-07,0.0000
14,SB,8.675096e-05,0.0001
16,SN,3.476822e-04,0.0003
3,BA,2.889683e-02,0.0289
6,CD,3.676709e-01,0.3677


In [ ]:
# P_value가 0.05 이하인 변수만 추출
Wilcoxon_var_table.loc[Wilcoxon_var_table['p_value'] <= 0.05, :].sort_values('p_value')

,variable,p_value,p_value_round
15,SI,0.000000e+00,0.0000
13,S,1.970075e-120,0.0000
7,K,1.599942e-50,0.0000
10,NA,5.842989e-36,0.0000
8,LI,2.692208e-09,0.0000
1,AL,2.754589e-07,0.0000
14,SB,8.675096e-05,0.0001
16,SN,3.476822e-04,0.0003
3,BA,2.889683e-02,0.0289


### 불량률

In [ ]:
# 각 변수에 대해 '정상 범위 데이터 개수', '이상 범위 데이터 개수', '전체 데이터 합계',
#               '정상 범위 데이터의 불량 개수', '이상 범위 데이터의 불량 개수', '불량 합계',
#               '정상 범위 데이터의 불량률', '이상 범위 데이터의 불량률'
# 에 대한 Table 생성

variable_7 = ['AL', 'BA', 'K', 'S', 'SB', 'SI', 'SN']

outlier_table = pd.DataFrame(np.zeros([8, len(variable_7)]), columns = variable_7,
                             index = ['정상 범위 데이터 개수', '이상 범위 데이터 개수', '전체 데이터 합계',
                                      '정상 범위 데이터의 불량 개수', '이상 범위 데이터의 불량 개수', '불량 합계',
                                      '정상 범위 데이터의 불량률', '이상 범위 데이터의 불량률'])

for v in variable_7 :
  temp1 = data_train2.loc[(IQR_outlier(data_train2).loc[0, v] <= data_train2[v]) &
                        (data_train2[v] <= IQR_outlier(data_train2).loc[1, v]), :]
  temp2 = data_train2.loc[(IQR_outlier(data_train2).loc[0, v] > data_train2[v]) | (data_train2[v] > IQR_outlier(data_train2).loc[1, v]), :]

  outlier_table.at['정상 범위 데이터 개수', v] = temp1.shape[0]
  outlier_table.at['이상 범위 데이터 개수', v] = temp2.shape[0]
  outlier_table.at['전체 데이터 합계', v] = temp1.shape[0] + temp2.shape[0]

  temp3 = temp1.loc[data_train2['Y_LABEL'] == 1, :]
  temp4 = temp2.loc[data_train2['Y_LABEL'] == 1, :]
  outlier_table.at['정상 범위 데이터의 불량 개수', v] = temp3.shape[0]
  outlier_table.at['이상 범위 데이터의 불량 개수', v] = temp4.shape[0]
  outlier_table.at['불량 합계', v] = temp3.shape[0] + temp4.shape[0]

  if temp1.shape[0] == 0 :
    temp5 = 0
  else :
    temp5 = np.round(temp3.shape[0] / temp1.shape[0] * 100, 3)
  outlier_table.at['정상 범위 데이터의 불량률', v] = temp5

  if temp2.shape[0] == 0 :
    temp6 = 0
  else :
    temp6 = np.round(temp4.shape[0] / temp2.shape[0] * 100, 3)
  outlier_table.at['이상 범위 데이터의 불량률', v] = temp6

outlier_table = outlier_table.T
outlier_table.nlargest(len(variable_17), columns = ['이상 범위 데이터의 불량률'])

<ipython-input-28-6733e88c8a8a>:4: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  Q1 = data.quantile(0.25)
<ipython-input-28-6733e88c8a8a>:5: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  Q3 = data.quantile(0.75)


,정상 범위 데이터 개수,이상 범위 데이터 개수,전체 데이터 합계,정상 범위 데이터의 불량 개수,이상 범위 데이터의 불량 개수,불량 합계,정상 범위 데이터의 불량률,이상 범위 데이터의 불량률
S,14088.0,7.0,14095.0,3017.0,7.0,3024.0,21.415,100.000
SI,12246.0,1849.0,14095.0,1937.0,1087.0,3024.0,15.817,58.789
K,10738.0,1058.0,11796.0,1879.0,468.0,2347.0,17.499,44.234
AL,12651.0,1444.0,14095.0,2626.0,398.0,3024.0,20.757,27.562
SB,11128.0,2967.0,14095.0,2287.0,737.0,3024.0,20.552,24.840
SN,12778.0,1317.0,14095.0,2702.0,322.0,3024.0,21.146,24.450
BA,10780.0,3315.0,14095.0,2258.0,766.0,3024.0,20.946,23.107


###18개 데이터 개별 통계 지표 및 속성값 시각화


In [ ]:
for i in range(2,len(train.columns)):
  # 데이터의 개별 속성값 관찰
  attribute_name = train.columns[i]

  # 적절한 요약 통계 지표를 계산합니다.
  mean_value = train[attribute_name].mean()
  median_value = train[attribute_name].median()
  variance_value = train[attribute_name].var()

  print("\n{}의 요약 통계:".format(attribute_name))
  print("평균: {:.2f}".format(mean_value))
  print("중앙값: {:.2f}".format(median_value))
  print("분산: {:.2f}".format(variance_value))

  # 시각화를 사용하여 데이터의 개별 속성값을 관찰
  plt.figure(figsize=(12, 6))
  plt.title("{}의 분포".format(attribute_name))
  plt.xlabel("값")
  plt.ylabel("빈도")
  data_train[attribute_name].hist(bins=30)
  plt.show()


###상관분석

In [ ]:
data_train.columns

In [ ]:
# 종속 변수(y_label)와 각 독립 변수 간의 상관 계수 계산
correlation_matrix = data_train.corr()

# 상관 계수 행렬에서 종속 변수와의 상관 계수만 추출
y_label_correlations_52 = correlation_matrix['Y_LABEL']

# 종속 변수와의 상관 계수를 내림차순으로 정렬하여 출력
sorted_correlations_52 = y_label_correlations_52.abs().sort_values(ascending=False)
print("종속 변수(y_label)와의 상관 계수:")
print(sorted_correlations_52)

# 상관 계수를 시각화하여 종속 변수에 미치는 영향 확인
plt.figure(figsize=(12, 6))
sns.barplot(x=sorted_correlations_52.index, y=sorted_correlations_52)
plt.title("독립 변수와 종속 변수(y_label) 간의 상관 계수")
plt.xlabel("독립 변수")
plt.ylabel("상관 계수")
plt.xticks(rotation=90)
plt.show()

In [ ]:
# 종속 변수(y_label)와 각 독립 변수 간의 상관 계수 계산
correlation_matrix = train.corr()

# 상관 계수 행렬에서 종속 변수와의 상관 계수만 추출
y_label_correlations_18 = correlation_matrix['Y_LABEL']

# 종속 변수와의 상관 계수를 내림차순으로 정렬하여 출력
sorted_correlations_18 = y_label_correlations_18.abs().sort_values(ascending=False)
print("종속 변수(y_label)와의 상관 계수:")
print(sorted_correlations)

# 상관 계수를 시각화하여 종속 변수에 미치는 영향 확인
plt.figure(figsize=(12, 6))
sns.barplot(x=sorted_correlations_18.index, y=sorted_correlations_18)
plt.title("독립 변수와 종속 변수(y_label) 간의 상관 계수")
plt.xlabel("독립 변수")
plt.ylabel("상관 계수")
plt.xticks(rotation=90)
plt.show()

In [ ]:
#test DATA feature만 가진 train DATA
train_yo = train[['AG',
       'CO', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V',
       'V40', 'ZN']].copy()

all_nan_or_zero_rows = train_yo[(train_yo.isna() | (train == 0)).all(axis=1)]
all_nan_or_zero_rows